In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [2]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
我们使用默认的 operator.add 或 + 归约器为 messages 状态键添加了注释，它总是将新消息附加到现有消息数组的末尾。
现在，为了支持替换现有消息，我们使用一个自定义的归约器函数为 messages 键添加注释，该函数会用相同 id 的消息进行替换，否则将其附加。
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [5]:
import os
prompt = """你是一个聪明的研究助手。使用搜索引擎查找信息。
你可以进行多次调用（一起或依次进行）。
只有在确定想要什么时才查找信息。
如果你在提出后续问题之前需要查找一些信息，你可以这样做！
"""
api_key = os.environ.get("MY_API_KEY")
base_url = os.environ.get("OPENAI_API_BASE")
model = ChatOpenAI(api_key=api_key, base_url=base_url,model="gpt-4-turbo") 
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [6]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='7b99e655-2801-4973-9996-2e10a95be47f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-19767b1e-232b-4430-8c39-be778f51b267-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'function': {'argum

In [7]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='7b99e655-2801-4973-9996-2e10a95be47f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-19767b1e-232b-4430-8c39-be778f51b267-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef4c9e0-806e-64cc-8001-caf969a1

In [8]:
abot.graph.get_state(thread).next

('action',)

In [9]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1722142671, \'localtime\': \'2024-07-27 21:57\'}, \'current\': {\'last_updated_epoch\': 1722141900, \'last_updated\': \'2024-07-27 21:45\', \'temp_c\': 14.2, \'temp_f\': 57.6, \'is_day\': 0, \'condition\': {\'text\': \'Patchy rain nearby\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/176.png\', \'code\': 1063}, \'wind_mph\': 12.5, \'wind_kph\': 20.2, \'wind_degree\': 271, \'wind_dir\': \'W\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.9, \'precip_mm\': 0.01, \'precip_in\': 0.0, \'humidity\': 85, \'clou

In [10]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='7b99e655-2801-4973-9996-2e10a95be47f'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-19767b1e-232b-4430-8c39-be778f51b267-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4CsvjpQTaGhPXGO3HGRh2gwJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Sa

In [11]:
abot.graph.get_state(thread).next

()

In [12]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='e0c04f0e-e3d9-4304-8af1-ae494eaa8563'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KZB4XsrPO4cXoi8rT3yERTMZ', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7d4da79e-3c69-4e08-a4f1-4b639122c56e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_KZB4XsrPO4cXoi8rT3yERTMZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KZB4XsrPO4cXoi8rT3yERTMZ', 'function': {'arguments

In [13]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments

In [14]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef4c9fd-d9d3-6b26-8001-1a5cebabab97

In [15]:
current_values = abot.graph.get_state(thread)

In [16]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})

In [17]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q',
  'type': 'tool_call'}]

In [18]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [19]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}


{'configurable': {'thread_id': '3',
  'thread_ts': '1ef4ca02-7c44-641a-8002-d92215956bdb'}}

In [20]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef4ca02-7c44-641a-8002-d92215956bdb'}

In [21]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1722143560, \'localtime\': \'2024-07-28 0:12\'}, \'current\': {\'last_updated_epoch\': 1722142800, \'last_updated\': \'2024-07-28 00:00\', \'temp_c\': 21.0, \'temp_f\': 69.9, \'is_day\': 0, \'condition\': {\'text\': \'Patchy rain nearby\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/176.png\', \'code\': 1063}, \'wind_mph\': 9.6, \'wind_kph\': 15.5, \'wind_degree\': 140, \'wind_dir\': \'SE\', \'pressure_mb\': 1015.0, \'pressure_in\': 29.96, \'precip_mm\': 0.01, \'precip_in\': 0.0, \'humidity\': 97, \'cloud\': 77, \'

In [22]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisian

In [23]:
to_replay = states[-3]

In [24]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef4c9fd-d9d3-6b26-8001-1a5cebabab97

In [25]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1722143701, \'localtime\': \'2024-07-27 22:15\'}, \'current\': {\'last_updated_epoch\': 1722142800, \'last_updated\': \'2024-07-27 22:00\', \'temp_c\': 22.5, \'temp_f\': 72.5, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.0, \'wind_kph\': 6.5, \'wind_degree\': 192, \'wind_dir\': \'SSW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.86, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 52, \'cloud\': 0, \'feelsli

In [26]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef4c9fd-d9d3-6b26-8001-1a5cebabab97

In [27]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [28]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='d9b0e27a-7d24-4f4b-b326-1b969d798f4e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 173, 'total_tokens': 195}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08cdee9f-828e-4855-9618-9103a00793a3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q'}], usage_metadata={'input_tokens': 173, 'output_tokens': 22, 'total_tokens': 195})]}


In [29]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_1wcfOxHjPgXFYhl61nP8uq5Q', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1722143701, \'localtime\': \'2024-07-27 22:15\'}, \'current\': {\'last_updated_epoch\': 1722142800, \'last_updated\': \'2024-07-27 22:00\', \'temp_c\': 22.5, \'temp_f\': 72.5, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.0, \'wind_kph\': 6.5, \'wind_degree\': 192, \'wind_dir\': \'SSW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.86, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 52, \'cloud\': 0, \'fee